In [6]:
spark.stop() fdfsfsf

# Checker for churn_nrt

In [1]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when '-' is removed from name, remove also this line and adapt imports 
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)

import time
start_time = time.time()

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
%matplotlib inline  
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import time
start_time = time.time()

from churn_nrt.src.utils.spark_session import get_spark_session

sc, spark, sql_context = get_spark_session("checker")
sc.setLogLevel('WARN')

from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf,countDistinct, \
        desc, asc, to_date, create_map, sum as sql_sum, substring, sort_array, split, month, dayofmonth, lag, round as sql_round
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType, DateType, FloatType
from pyspark.sql.functions import array, regexp_extract
from itertools import chain
import argparse

import csv
import re
import subprocess
import sys
import time
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.functions import concat_ws, date_format, from_unixtime, \
    length, lit, lower, lpad, month, regexp_replace, translate, udf, unix_timestamp, year, when, upper
from pyspark.sql.utils import AnalysisException
from collections import Counter
from pyspark.sql.types import StringType
from pyspark.sql.window import Window

In [2]:
from churn_nrt.src.utils.hdfs_functions import run_cmd


def count_files_in_path(path_):

    cmd = "hdfs dfs -ls {} | wc -l".format(path_)
    output = subprocess.check_output(cmd, shell=True).strip().split('\n')
    num_files = int(output[0]) - 1 # the first entry is the summary
    return num_files

    
count_udf = udf(count_files_in_path, IntegerType())



def investigate_duplicates(path_):
    
    if not path_: return "path is none"
    
    num_files = count_files_in_path(path_)
   
    if num_files == 400:
        cmd = "hdfs dfs -ls -R {}".format(path_)
        output = subprocess.check_output(cmd, shell=True).strip().split('\n')

        pattern1 = re.search("^.+\/part-(\d{5})-(.{8})-.+parquet$", output[-2]).group(2)
        print(pattern1)
        print(path_)
        return "hdfs dfs -rm {}{}*".format(os.path.join(path_, "*"), pattern1)
    else:
        return "hdfs dfs -rm -r {}".format(path_)
    
hint_udf = udf(investigate_duplicates, StringType())


def get_user(path_):
    if not path_: return "path is none"

    cmd = "hdfs dfs -ls {}".format(path_)
    output = subprocess.check_output(cmd, shell=True).strip().split('\n')
    try:
        username = re.search("^(.+) (.+) (.+) bdpmd(.+)parquet$", output[-2]).group(3)
    except:
        return "problemilla"
    return username.strip()

user_udf = udf(get_user, StringType())

import numpy as np
from scipy.stats import skew

def add_scores_metrics(scores_list):

    avg_score = np.mean(scores_list)
    sd_score = np.std(scores_list)
    sk_score = skew(scores_list)
    #ku_score = score_rdd.map(lambda x: pow((x - avg_score) / sd_score, 4.0)).mean()
    min_score = np.min(scores_list)
    max_score = np.max(scores_list)
    return "{0:.2f} | {1:.2f} | {2:.2f} | {3:.2f} | {4:.2f}".format(avg_score, sd_score, sk_score, min_score, max_score) 
    
add_scores_udf = udf(add_scores_metrics, StringType())


def check_model_scores(model_, level="msisdn", check_freq=None, from_month=None, from_year=None):
    '''
    check_freq: None --> no dot check freq
    '''
    
    df_scores = spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name={}".format(model_))
    
    if from_year:
        df_scores = df_scores.where(col("year")>=from_year)
    if from_month:
        df_scores = df_scores.where(col("month")>=from_month)

    df_scores = (df_scores
                  .withColumn("deliv_partition", concat(col("year"), lpad(col("month"), 2, '0'), lpad(col("day"), 2, '0')))
                  .select("executed_at", "predict_closing_date", "deliv_partition", level, "scoring")
                  .groupby("executed_at", "predict_closing_date", "deliv_partition")
                   .agg(sql_count("*").alias(model_), countDistinct(level).alias("distinct"), add_scores_udf(collect_list("scoring")).alias("scores_metrics"))
                )
                

    df_scores = df_scores.withColumn("distinct_verified", when(col(model_) - col("distinct") == 0, "yes").otherwise("no"))

    df_params = spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name={}".format(model_)).select("executed_at", "training_closing_date")
    df_scores = df_scores.join(df_params, on=["executed_at"], how="outer")
    df_scores = df_scores.select("executed_at", "training_closing_date", "predict_closing_date", "deliv_partition", model_, "distinct_verified", "scores_metrics")
    
    if check_freq:
        
        my_window = Window.partitionBy().orderBy("deliv_partition")
        diff_days_udf = udf(lambda start_date,end_date: (dt.datetime.strptime(end_date, "%Y%m%d") - dt.datetime.strptime(start_date, "%Y%m%d")).days if end_date and start_date else -1000, IntegerType())

        df_scores = df_scores.withColumn("prev_value", lag(df_scores["deliv_partition"]).over(my_window))
        df_scores = df_scores.withColumn("diff", when(col("prev_value").isNotNull() & col("deliv_partition").isNotNull(), diff_days_udf(col("prev_value"),col("deliv_partition"))).otherwise(None)).drop("prev_value")
        df_scores = df_scores.withColumn("freq_verified", when(col("diff")==check_freq, "yes").otherwise("no"))

        df_scores = df_scores.withColumn("distinct|freq", concat_ws("|", col("distinct_verified"), "freq_verified")).drop("distinct_verified", "freq_verified")
    else:
        df_scores = df_scores.withColumn("distinct", col("distinct_verified")).drop("distinct_verified")
        
    

    
    my_window = Window.partitionBy().orderBy("deliv_partition")
    rel_inc_udf = udf(lambda prev,curr: np.float(100.0*(curr - prev)/prev) if prev and curr else 0, FloatType())
    df_scores = df_scores.withColumn("prev_value", lag(df_scores[model_]).over(my_window))
    df_scores = df_scores.withColumn("rel_inc", when(col("prev_value").isNotNull() & col(model_).isNotNull(), rel_inc_udf(col("prev_value"),col(model_))).otherwise(0)).drop("prev_value")
    df_scores = df_scores.withColumn("rel_inc", sql_round(df_scores["rel_inc"],2))

            
    df_scores = df_scores.withColumn("training_closing_date", regexp_extract(col('training_closing_date'), '(\w+)to(\w+)', 1)).withColumnRenamed("training_closing_date", "tr_date").withColumnRenamed("predict_closing_date", "tt_date")
    
    
    

    
    df_scores = df_scores.withColumn("path_",  concat(lit("/data/attributes/vf_es/model_outputs/model_scores/model_name={}".format(model_)), lit("/year="), col("deliv_partition").substr(0, 4),
                                                     lit("/month="), col("deliv_partition").substr(5, 2).cast(IntegerType()).cast(StringType()),
                                                     lit("/day="), col("deliv_partition").substr(7, 2).cast(IntegerType()).cast(StringType()))).drop("diff")
                                     
    df_scores = df_scores.withColumn("user", user_udf(col("path_")))
    df_scores = df_scores.withColumnRenamed("scores_metrics", " avg |  std |  sk |  min |  max")
    df_scores = df_scores.withColumnRenamed(model_, "volume")
    

    
    return df_scores


# def list_all_new_directories(root_path, from_date):
    
#     if not from_date:
#         print("from_date argument must not be None")
#         return None
    
#     cmd = "hdfs dfs -ls -R {} | grep drwx | grep 'day=' | awk '$6 > "2020-08-01"'".format(root_path,path_)
#     output = subprocess.check_output(cmd, shell=True).strip().split('\n')
#     print output
#     return output
    
# list_all_new_directories("/data/udf/vf_es/churn_nrt/", from_date="20200801")

# Checking standard modules

In [25]:
import datetime as dt
import re
from churn_nrt.src.utils.hdfs_functions import list_all_new_directories
    
    
def check_churn_nrt_partition(path_):
    try:
        level = levels_dict[module_name]
        count_ = spark.read.load(path_).select(level).count()
        distinct_ = spark.read.load(path_).select(level).distinct().count()

        if count_ == distinct_:
            #print("No issues found on module {}".format(path_))
            return True
            
    except Exception as e:
        print(" !! Exception analyzing module {}".format(path_))
        print(e)
        return False
    print("Issues found on module {} - Hint {}".format(path_, investigate_duplicates(path_)))
    return False
    
from_date="20200101"
to_date="20200331"    
new_dirs = list_all_new_directories("/data/udf/vf_es/churn_nrt/", from_date=from_date, to_date=to_date)

print("{} directories created from date {} to date {}".format(len(new_dirs), from_date, to_date))

#dd = '/data/udf/vf_es/churn_nrt/navcomp/15/year=2020/month=5/day=14'
#list_all_new_directories
unknown_modules = []
correct_paths=[]
incorrect_paths=[]
    
for dd in new_dirs:
    try:      
        module_name = re.match("^\/data\/udf\/vf_es\/churn_nrt\/(.*)\/year=.*$", dd).group(1)
    except:
        print("erorr extracting module_name with dd '{}'".format(dd))


    if module_name in levels_dict:
        res = check_churn_nrt_partition(dd)
        if res:
            correct_paths.append(dd)
        else:
            incorrect_paths.append(dd)
    else:
        unknown_modules.append(module_name)
    
print("unknown module names - checked was not run over them:")
print(set(unknown_modules))

if incorrect_paths:
    print("problemas found on following modules:")
    for dd in incorrect_paths:
        print("Issues found on module {} - Hint {}".format(dd, investigate_duplicates(dd)))
else:
    print("no found problems")


[hdfs_functions] list_all_new_directories | Running command
hdfs dfs -ls -R /data/udf/vf_es/churn_nrt/ | grep drwx | grep 'day=' | awk '$6 >= "2020-01-01"' | awk '$6 <= "2020-03-31"'
1376 directories created from date 20200101 to date 20200331
Issues found on module /data/udf/vf_es/churn_nrt/spinners/year=2019/month=11/day=14 - Hint hdfs dfs -rm -r /data/udf/vf_es/churn_nrt/spinners/year=2019/month=11/day=14
unknown module names - checked was not run over them:
set(['fix_port/77', 'mob_port/10', 'mob_port/30', 'fix_port/70', 'mob_port/14', 'fix_port/30', 'fix_port/10', 'trigger_myvf/app/v2_dup', 'mob_port/98', 'zhilabs_dsl_car', 'fix_port/98', 'dsl_car', 'dsl_data', 'trigger_myvf/web/v1', 'trigger_myvf/web/v2', 'mob_port/29', 'mob_port/60', 'mob_port/26', 'fix_port/56', 'mob_port/25', 'fix_port/60', 'trigger_myvf/app/v1', 'tnps', 'trigger_myvf/app/v3', 'trigger_social', 'fix_port/25', 'fix_port/26', 'myvf/app/v1', 'scores', 'fix_port/29', 'ccc_nif', 'hfc_data', 'trigger_service', 'sfid

{'ftth_car',
 'ftth_car/critical',
 'ftth_car/warning',
 'ftth_data',
 'ftth_data/critical',
 'ftth_data/critical/critical',
 'ftth_inc_data',
 'ftth_inc_data/critical',
 'scores',
 'trigger_myvf/web/v3',
 'trigger_navcomp',
 'zhilabs_ftth_car',
 'zhilabs_ftth_car/None',
 'zhilabs_ftth_car/critical',
 'zhilabs_ftth_car/warning'}

In [10]:
levels_dict = {"customer" : "num_cliente",
               "service"  : "msisdn",
               "customer_base" : "msisdn",
               "customer_additional/90" : "nif_cliente",
               "billing" : "num_cliente",
               "navcomp*" : "msisdn",
                "billing" : "num_cliente",
               "callscomp/*" : "msisdn",
               "navcomp_adv" : "msisdn",
                "callscomp_adv" : "msisdn",
               "tgs" : "msisdn",
               "tickets" : "nif_cliente",
               "spinners" : "nif_cliente",
               "reimbursements" : "NIF_CLIENTE",
               "orders_sla" : "nif_cliente",
               'ccc/msisdn' : "msisdn",
               'ccc/nif' : "nif_cliente",
               'mob_port/*' : 'msisdn',
               'myvf/app/2' : "msisdn",
               'myvf/web/3' : "msisdn",
              }


module_name = "mob_ffffport/16"

import re


def __get_level(module_name)
    level = [v for k,v in levels_dict.items() if re.compile(k).match(module_name)]
    if level:
        return level[0]
    return None




[]

In [5]:




# INPUT SECTION

MODULES = [
    'mob_port/15',
     'customer_base',
     'customer',
     'service',
     'tgs',
     'ccc/msisdn',
     'navcomp/7',
     'navcomp/15',
     'navcomp/30',
     'orders_sla',
    'billing',
     'customer_additional/90',
    'reimbursements',
    'spinners',
    'navcomp_adv',
     'callscomp/7',
     'callscomp/15',
     'callscomp/30',
     'myvf/app/2',
   #  'myvf/web/3',
    'tickets'
          ]

YEARS = ["2020"]



# DO NOT EDIT BELOW THIS POINT!

ALL_MODULES = ['tgs',
 'tickets',
 'navcomp_adv'              
 'navcomp/15',
 'navcomp/7',
 'orders_sla',
 'customer',
 'service',
 'billing',
 'customer_base',
 'customer_additional/90',
 'reimbursements',
 'spinners',
 'ccc/msisdn',
 'ccc/nif' ]
ALL_YEARS = ["2020", "2019"]

start_time = time.time()

levels_dict = {"customer" : "num_cliente",
               "service"  : "msisdn",
               "customer_base" : "msisdn",
               "customer_additional/90" : "nif_cliente",
               "billing" : "num_cliente",
               "navcomp/7" : "msisdn",
               "navcomp/15" : "msisdn",
                "navcomp/30" : "msisdn",
                "billing" : "num_cliente",
               "callscomp/7" : "msisdn",
               "callscomp/15" : "msisdn",
                "callscomp/30" : "msisdn",
               "navcomp_adv" : "msisdn",
                "callscomp_adv" : "msisdn",
               "tgs" : "msisdn",
               "tickets" : "nif_cliente",
               "spinners" : "nif_cliente",
               "reimbursements" : "NIF_CLIENTE",
               "orders_sla" : "nif_cliente",
               'ccc/msisdn' : "msisdn",
               'ccc/nif' : "nif_cliente",
               'mob_port/15' : 'msisdn',
               'myvf/app/2' : "msisdn",
               'myvf/web/3' : "msisdn",
              }
 
ALL_MODULES = levels_dict.keys()
ALL_YEARS = ["2020", "2019"]

for module_name in MODULES:
    
    for year_ in YEARS:

        print(" ** MODULE /data/udf/vf_es/churn_nrt/{}/year={}/ ** ".format(module_name, year_))

        try:
            df_ = (spark.read.load("/data/udf/vf_es/churn_nrt/{}/year={}/".format(module_name, year_)).groupby("month", "day")
                      .agg(sql_count("*").alias("count"), 
                           countDistinct(levels_dict[module_name]).alias("countDis"))
                      .withColumn("verified", when(col("count") - col("countDis") == 0, "yes").otherwise("no"))
                      .where(col("verified")=="no")
                      .withColumn("hdfs_path", concat(lit("/data/udf/vf_es/churn_nrt/{}/year={}/month=".format(module_name, year_)), col("month"), lit("/day="), col("day")))
                      .withColumn("num_files", count_udf(col("hdfs_path")))
                      .withColumn("suggested_action", when(col("verified")=="no", hint_udf(col("hdfs_path"))).otherwise("-"))
                      .withColumn("user", user_udf(col("hdfs_path")))  
                   )

            if df_.count() == 0:
                   print("No issues found on module /data/udf/vf_es/churn_nrt/{}/year={}/".format(module_name, year_))
            else:
                df_.show(truncate=False)
        except Exception as e:
            print(" !! Exception analyzing module /data/udf/vf_es/churn_nrt/{}/year={}/ ** ".format(module_name, year_))
            print(e)

         
         

            
print("Elapsed time {} minutes".format( (time.time() - start_time)/60.0))

 ** MODULE /data/udf/vf_es/churn_nrt/mob_port/15/year=2020/ ** 
No issues found on module /data/udf/vf_es/churn_nrt/mob_port/15/year=2020/
 ** MODULE /data/udf/vf_es/churn_nrt/customer_base/year=2020/ ** 
No issues found on module /data/udf/vf_es/churn_nrt/customer_base/year=2020/
 ** MODULE /data/udf/vf_es/churn_nrt/customer/year=2020/ ** 
+-----+---+--------+--------+--------+-----------------------------------------------------------+---------+---------------------------------------------------------------------------+--------+
|month|day|count   |countDis|verified|hdfs_path                                                  |num_files|suggested_action                                                           |user    |
+-----+---+--------+--------+--------+-----------------------------------------------------------+---------+---------------------------------------------------------------------------+--------+
|5    |21 |39339644|9834911 |no      |/data/udf/vf_es/churn_nrt/customer/ye

No issues found on module /data/udf/vf_es/churn_nrt/navcomp_adv/year=2020/
 ** MODULE /data/udf/vf_es/churn_nrt/callscomp/7/year=2020/ ** 
No issues found on module /data/udf/vf_es/churn_nrt/callscomp/7/year=2020/
 ** MODULE /data/udf/vf_es/churn_nrt/callscomp/15/year=2020/ ** 
No issues found on module /data/udf/vf_es/churn_nrt/callscomp/15/year=2020/
 ** MODULE /data/udf/vf_es/churn_nrt/callscomp/30/year=2020/ ** 
No issues found on module /data/udf/vf_es/churn_nrt/callscomp/30/year=2020/
 ** MODULE /data/udf/vf_es/churn_nrt/myvf/app/2/year=2020/ ** 
No issues found on module /data/udf/vf_es/churn_nrt/myvf/app/2/year=2020/
 ** MODULE /data/udf/vf_es/churn_nrt/tickets/year=2020/ ** 
+-----+---+-------+--------+--------+----------------------------------------------------------+---------+----------------------------------------------------------------------------------+---------+
|month|day|count  |countDis|verified|hdfs_path                                                 |num_files|s

# En producción

# Daily triggers: myvf_app, myvf_web & navcomp

In [11]:
print(" ")

for model_ in ["myvf_app", "myvf_web",  
    "triggers_navcomp"]:
    start_time = time.time()
    print("* * * * * * * * * {} * * * * * * * * *".format(model_))
    df_scores = check_model_scores(model_, check_freq=1, from_month=8, from_year=2020)
    df_scores.sort(desc("deliv_partition"), desc("executed_at"), desc("tt_date"), desc("tr_date")).show(10, truncate=False)
    print("Elapsed time: {} minutes".format( (time.time() - start_time)/60.0))
    print(" ")

 
* * * * * * * * * myvf_app * * * * * * * * *
+-------------------+--------+--------+---------------+------+--------------------------------+-------------+-------+----------------------------------------------------------------------------------------------+---------+
|executed_at        |tr_date |tt_date |deliv_partition|volume| avg |  std |  sk |  min |  max |distinct|freq|rel_inc|path_                                                                                         |user     |
+-------------------+--------+--------+---------------+------+--------------------------------+-------------+-------+----------------------------------------------------------------------------------------------+---------+
|2020-08-20 13:16:46|20200730|20200815|20200820       |83812 |0.03 | 0.03 | 6.80 | 0.01 | 0.81|yes|yes      |-3.17  |/data/attributes/vf_es/model_outputs/model_scores/model_name=myvf_app/year=2020/month=8/day=20|csanc109 |
|2020-08-18 22:27:11|20200727|20200812|20200819       |86556 

# Weekly triggers: Zhilabs, servicios and tnps


In [6]:

level_dict = {"zhilabs_ftth" : "client_id", "triggers_ml" : "nif",  "trigger_tnps" : "msisdn"}


for model_ in ["zhilabs_ftth", "triggers_ml", "trigger_tnps"]:  
    print("* * * * * * * * * {} * * * * * * * * *".format(model_))
    df_scores = check_model_scores(model_, level=level_dict[model_], check_freq=None)
    df_scores.sort(desc("deliv_partition"), desc("executed_at"), desc("tt_date"), desc("tr_date")).show(10, truncate=False)
    print(" ")

* * * * * * * * * zhilabs_ftth * * * * * * * * *
+-------------------+--------+--------+---------------+------------+---------------------------------+--------+----------+----------+--------------------------------------------------------------------------------------------------+---------+
|executed_at        |tr_date |tt_date |deliv_partition|zhilabs_ftth| avg |  std  |  sk |  min |  max |distinct|prev_value|rel_inc   |path_                                                                                             |user     |
+-------------------+--------+--------+---------------+------------+---------------------------------+--------+----------+----------+--------------------------------------------------------------------------------------------------+---------+
|2020-08-10 03:16:49|20200702|20200806|20200812       |435363      |0.03 | 0.02 | 2.17 | 0.01 | 0.19 |yes     |458489    |-5.0439596|/data/attributes/vf_es/model_outputs/model_scores/model_name=zhilabs_ftth/year=2020/month

KeyboardInterrupt: 

# Twice-a-month triggers - social

In [8]:
print(" ")

for model_ in ["trigger_social"]:
    print("* * * * * * * * * {} * * * * * * * * *".format(model_))
    df_scores = check_model_scores(model_)
    df_scores.sort(desc("deliv_partition"), desc("executed_at"), desc("tt_date"), desc("tr_date")).show(10, truncate=False)
    print(" ")

 
* * * * * * * * * trigger_social * * * * * * * * *
+-------------------+--------+--------+---------------+--------------+--------+----------------------------------------------------------------------------------------------------+--------+
|executed_at        |tr_date |tt_date |deliv_partition|trigger_social|distinct|path_                                                                                               |user    |
+-------------------+--------+--------+---------------+--------------+--------+----------------------------------------------------------------------------------------------------+--------+
|2020-08-09 00:01:40|20200705|20200805|20200810       |157213        |yes     |/data/attributes/vf_es/model_outputs/model_scores/model_name=trigger_social/year=2020/month=8/day=10|csanc109|
|2020-07-23 11:55:41|20200619|20200720|20200725       |136729        |yes     |/data/attributes/vf_es/model_outputs/model_scores/model_name=trigger_social/year=2020/month=7/day=25|asaezco

# Triggers in test phase: Price sens

In [6]:
import datetime as dt
from churn_nrt.src.utils.date_functions import move_date_n_days
from churn_nrt.src.utils.hdfs_functions import get_partitions_path_range

PATH_CDR_RAW = "/data/raw/vf_es/mediated_cdr/NAVAJO/1.0/parquet/"


def get_last_date(spark):
    
    closing_day = dt.datetime.today().strftime("%Y%m%d")
    starting_day = move_date_n_days(closing_day, n=-30)    

    for try_ in [0,1,2]:

        cdr_paths = get_partitions_path_range(PATH_CDR_RAW, starting_day, closing_day)[::-1]
        print(starting_day, closing_day)

        for path_ in cdr_paths:

            print(path_)

            callscomp_last_date = (spark.read.parquet(path_).where(col("CDSENLLA") == "OL")
                .select(sql_max(col('DTINILLA')).alias('last_date')).rdd.first()['last_date'])

            return int(callscomp_last_date)

        closing_day = move_date_n_days(starting_day, n=-1)
        starting_day = move_date_n_days(starting_day, n=-30)

    return "None"

get_last_date(spark)

('20200721', '20200820')
/data/raw/vf_es/mediated_cdr/NAVAJO/1.0/parquet/year=2020/month=8/day=20


20200820

In [35]:
callscomp_last_date.select("mydate").show()

NameError: name 'callscomp_last_date' is not defined

In [28]:
print(" ")
for model_ in ["prop_virgin", "price_sens_high_potential", "price_sens_glob_pot_classifier"]:
    print("* * * * * * * * * {} * * * * * * * * *".format(model_))
    df_scores = check_model_scores(model_)
    df_scores.sort(desc("deliv_partition"), desc("executed_at"), desc("tt_date"), desc("tr_date")).show(10, truncate=False)
    print(" ")

 
* * * * * * * * * prop_virgin * * * * * * * * *
+-------------------+--------+--------+---------------+------+--------------------------------+--------+-------+-------------------------------------------------------------------------------------------------+--------+
|executed_at        |tr_date |tt_date |deliv_partition|volume| avg |  std |  sk |  min |  max |distinct|rel_inc|path_                                                                                            |user    |
+-------------------+--------+--------+---------------+------+--------------------------------+--------+-------+-------------------------------------------------------------------------------------------------+--------+
|2020-08-19 12:43:46|20200630|20200730|20200820       |25000 |0.85 | 0.01 | 0.77 | 0.84 | 0.89|yes     |-72.1  |/data/attributes/vf_es/model_outputs/model_scores/model_name=prop_virgin/year=2020/month=8/day=20|csanc109|
|2020-07-29 08:33:51|20200710|20200726|20200729       |89590 |0.03 | 0

# Triggers data - data template modules after model_classes

In [9]:
for trig in ["trigger_myvf/app/v2/", "trigger_myvf/web/v3/",  
             "trigger_navcomp/", 
             "trigger_social"
            ]:
    print("* * * * * * * * * {} * * * * * * * * *".format("/data/udf/vf_es/churn_nrt/"+trig))
    df_ = (spark.read.load("/data/udf/vf_es/churn_nrt/"+trig).groupby("year", "month", "day")
          .agg(sql_count("*").alias("count"), 
               countDistinct("msisdn").alias("countDis"))
          .withColumn("distinct", when(col("count") - col("countDis") == 0, "yes").otherwise("no"))
          .where(col("distinct")=="no").withColumn("hdfs_path", concat(lit("/data/udf/vf_es/churn_nrt/"+trig+"year="), col("year"), lit("/month="), col("month"), lit("/day="), col("day")))
          .withColumn("num_files", count_udf(col("hdfs_path")))
          .withColumn("suggested_action", hint_udf(col("hdfs_path")))
          .withColumn("user", user_udf(col("hdfs_path")))  
          .sort(asc("month"), asc("day")))
    if df_.count() == 0:
        print("   No issues found on module /data/udf/vf_es/churn_nrt/"+trig)
        print("")
        print("")
    else:
        df_.show(truncate=False)
        print(" ")
        print(" ")


* * * * * * * * * /data/udf/vf_es/churn_nrt/trigger_myvf/app/v2/ * * * * * * * * *
   No issues found on module /data/udf/vf_es/churn_nrt/trigger_myvf/app/v2/


* * * * * * * * * /data/udf/vf_es/churn_nrt/trigger_myvf/web/v3/ * * * * * * * * *
   No issues found on module /data/udf/vf_es/churn_nrt/trigger_myvf/web/v3/


* * * * * * * * * /data/udf/vf_es/churn_nrt/trigger_navcomp/ * * * * * * * * *
   No issues found on module /data/udf/vf_es/churn_nrt/trigger_navcomp/


* * * * * * * * * /data/udf/vf_es/churn_nrt/trigger_social * * * * * * * * *
   No issues found on module /data/udf/vf_es/churn_nrt/trigger_social




In [10]:
for trig in ["trigger_virgin"]:
    print("* * * * * * * * * {} * * * * * * * * *".format("/data/udf/vf_es/churn_nrt/"+trig))
    df_ = (spark.read.load("/data/udf/vf_es/churn_nrt/"+trig).groupby("year", "month", "day")
          .agg(sql_count("*").alias("count"), 
               countDistinct("msisdn").alias("countDis"))
          .withColumn("distinct", when(col("count") - col("countDis") == 0, "yes").otherwise("no"))
          .where(col("distinct")=="no").withColumn("hdfs_path", concat(lit("/data/udf/vf_es/churn_nrt/"+trig+"year="), col("year"), lit("/month="), col("month"), lit("/day="), col("day")))
          .withColumn("num_files", count_udf(col("hdfs_path")))
          .withColumn("suggested_action", hint_udf(col("hdfs_path")))
          .sort(asc("month"), asc("day")))
    if df_.count() == 0:
        print("   No issues found on module /data/udf/vf_es/churn_nrt/"+trig)
        print("")
        print("")
    else:
        df_.show(truncate=False)
        print(" ")
        print(" ")


* * * * * * * * * /data/udf/vf_es/churn_nrt/trigger_virgin * * * * * * * * *
   No issues found on module /data/udf/vf_es/churn_nrt/trigger_virgin




# Old Delivery Churn

In [7]:
df_onlymob = (spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_onlymob")
              .select("executed_at", "predict_closing_date").groupby("executed_at", "predict_closing_date").agg(sql_count("*").alias("onlymob")))
df_onlymob_params = spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=churn_preds_onlymob").select("executed_at", "training_closing_date")
df_onlymob = df_onlymob.join(df_onlymob_params, on=["executed_at"], how="outer")
df_onlymob = df_onlymob.select("training_closing_date", "predict_closing_date", "onlymob")


df_mobandfbb = (spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_mobileandfbb")
                .select("executed_at", "predict_closing_date").groupby("executed_at", "predict_closing_date").agg(sql_count("*").alias("mobileandfbb")))

df_mobandfbb_params = spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=churn_preds_mobileandfbb").select("executed_at", "training_closing_date")
df_mobandfbb = df_mobandfbb.join(df_mobandfbb_params, on=["executed_at"], how="outer")
df_mobandfbb = df_mobandfbb.select("training_closing_date", "predict_closing_date", "mobileandfbb")


df_others = (spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_others")
              .select("executed_at", "predict_closing_date").groupby("executed_at", "predict_closing_date").agg(sql_count("*").alias("others")))
df_others_params = spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=churn_preds_others").select("executed_at", "training_closing_date")
df_others = df_others.join(df_others_params, on=["executed_at"], how="outer")
df_others = df_others.select("training_closing_date", "predict_closing_date", "others")


df_fbb = (spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_fbb")
            .select("executed_at", "predict_closing_date").groupby("executed_at", "predict_closing_date").agg(sql_count("*").alias("fbb")))
df_fbb_params = spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=churn_preds_fbb").select("executed_at", "training_closing_date")
df_fbb = df_fbb.join(df_fbb_params, on=["executed_at"], how="outer")
df_fbb = df_fbb.select("training_closing_date", "predict_closing_date", "fbb")


df_deliv = (spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=delivery_churn")
            .withColumn("deliv_date", concat(col("year"), lpad(col("month"), 2, '0'), lpad(col("day"), 2, '0')))
            .select("executed_at", "predict_closing_date", "deliv_date").groupby("deliv_date", "executed_at", "predict_closing_date").agg(sql_count("*").alias("deliv")))
df_deliv_params = spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=delivery_churn").select("executed_at", "training_closing_date")
df_deliv = df_deliv.join(df_deliv_params, on=["executed_at"], how="outer")
df_deliv = df_deliv.select("training_closing_date", "predict_closing_date", "deliv", "deliv_date")

df_all = df_onlymob.join(df_mobandfbb, on=["predict_closing_date", "training_closing_date"], how="outer")
df_all = df_all.join(df_others, on=["predict_closing_date", "training_closing_date"], how="outer")
df_all = df_all.join(df_fbb, on=["predict_closing_date", "training_closing_date"], how="outer")
df_all = df_all.join(df_deliv, on=["predict_closing_date", "training_closing_date"], how="outer")

df_all.sort(desc("predict_closing_date"), desc("training_closing_date")).show(100)


+--------------------+---------------------+-------+------------+------+-------+-------+----------+
|predict_closing_date|training_closing_date|onlymob|mobileandfbb|others|    fbb|  deliv|deliv_date|
+--------------------+---------------------+-------+------------+------+-------+-------+----------+
|            20200714|   20200514to20200514|1173944|     3881092|629356|2198217|5684392|  20200724|
|            20200707|   20200507to20200507|1179201|     3884475|631308|2200541|5694984|  20200717|
|            20200630|   20200430to20200430|1189589|     3886454|633719|2199639|5709762|  20200717|
|            20200621|   20200421to20200421|1192183|     3886963|634865|2204797|5714011|  20200703|
|            20200614|   20200414to20200414|1192410|     3885676|635506|2202805|5713592|  20200626|
|            20200607|   20200407to20200407|1193832|     3886854|635990|2205247|5716676|  20200619|
|            20200531|   20200331to20200331|1193286|     3888998|636563|2206687|5718847|  20200612|
